In [ ]:
import os
from google.colab import drive

# force mount just in case
drive.mount('/content/drive', force_remount=True)
root = '/content/drive/MyDrive/NeuroMagos_Runs'
os.makedirs(root, exist_ok=True)
print(f"saving to {root}")

# get code
if os.path.exists('NeuroMagos'):
    !cd NeuroMagos && git pull
else:
    !git clone https://github.com/ChandraguptSharma07/NeuroMagos.git

%cd NeuroMagos

# get data
if not os.path.exists('Synapse_Dataset'):
    !pip install -r requirements.txt
    !gdown --id 1lsh4VjvuTJKzmpzrwIKsr6QfveoGlaX6 -O data.zip
    !unzip -q data.zip

In [ ]:
# run this bad boy
# YOU: 
# !python train.py --fold 0 --k_folds 10 --model resnet --epochs 100 --save_dir {root}/F0_ResNet
# !python train.py --fold 0 --k_folds 10 --model cnn1d  --epochs 100 --save_dir {root}/F0_CNN1D

!python train.py --fold 0 --k_folds 10 --model resnet --epochs 100 --save_dir {root}/TestpRun

In [ ]:
import os, shutil, glob

print("saving the files...")
rescue_path = f"{root}/MODEL-DATA"
os.makedirs(rescue_path, exist_ok=True)

found = 0
# search everything in /content (skipping drive itself)
for r, d, f in os.walk('/content'):
    if '/content/drive' in r: continue
    
    for file in f:
        # save pth, csv, logs
        if any(file.endswith(x) for x in ['.pth', '.csv', '.png', '.txt']):
            # only keep relevant ones
            if any(k in file for k in ['best', 'last', 'hist', 'report', 'conf']):
                try:
                    src = os.path.join(r, file)
                    dst = os.path.join(rescue_path, file)
                    shutil.copy2(src, dst)
                    print(f"gotcha: {file}")
                    found += 1
                except: pass

print(f"saved {found} files to {rescue_path}")